In [132]:
import requests
import json
import time
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def rumble(line, cell=None):
    if cell is None:
        data = line
    else:
        data = cell
    
    
    start = time.time()                                                         
    response = json.loads(requests.post(server, data=data, params = {"result-size" : 1000}).text)                   
    end = time.time()                                                              
    print("Took: %s s" % (end - start))
    if 'warning' in response:
        print(json.dumps(response['warning']))
    if 'values' in response:
        for e in response['values']:
            print(json.dumps(e))
            print()
        #with open('matches_id.json', 'w', encoding="utf-8") as outfile:
            #json.dump(response['values'], outfile, ensure_ascii = False)
    elif 'error-message' in response:
        return response['error-message']
    else:
        return response

In [133]:
def rumble_return(line, cell=None):
    if cell is None:
        data = line
    else:
        data = cell
    
    
    start = time.time()                                                         
    response = json.loads(requests.post(server, data=data, params = {"result-size" : 10000}).text)                   
    end = time.time()                                                              
    #print("Took: %s s" % (end - start))
    if 'warning' in response:
        print(json.dumps(response['warning']))
    if 'values' in response:
        return json.dumps(response['values'])
        
        with open('matches_id_UT.json', 'w', encoding="utf-8") as outfile:
            json.dump(response['values'], outfile, ensure_ascii=False)
    elif 'error-message' in response:
        return response['error-message']
    else:
        return response

In [134]:
server = 'http://localhost:8001/jsoniq'
neo_uri = "neo4j+s://00e145e7.databases.neo4j.io:7687"
neo_user = "soccer_analytics"
neo_pass = "night-candle-miracle-nickel-declare-32"


## New

Simple scripts to load data to neo4j. In order to run it, rumbleDB is needed

### Loading players

In [135]:
from neo4j import GraphDatabase
import logging
from neo4j.exceptions import ServiceUnavailable
import streamlit as st
class App:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        # Don't forget to close the driver connection when you are finished with it
        self.driver.close()

    def create_player(self, query):
        with self.driver.session() as session:
            # Write transactions allow the driver to handle retries and transient errors
            result = session.write_transaction(
                self._create_and_return_player, query)
            for row in result:
                print("Created player: {p1}".format(p1=row))

    @staticmethod
    def _create_and_return_player(tx, query):
        # To learn more about the Cypher syntax, see https://neo4j.com/docs/cypher-manual/current/
        # The Reference Card is also a good resource for keywords https://neo4j.com/docs/cypher-refcard/current/
        
        result = tx.run(query)

        try:
            return [{"": row["x"]["name"]}
                    for row in result]
        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise
    
    def create_relationship(self, query):
        with self.driver.session() as session:
            # Write transactions allow the driver to handle retries and transient errors
            result = session.write_transaction(
                self._create_and_return_relationship, query)
            

    @staticmethod
    def _create_and_return_relationship(tx, diz):

        result = tx.run(query)
        try:
            return [{"row": row}
                    for row in result]

        # Capture any errors along with the query and data for traceability
        except ServiceUnavailable as exception:
            logging.error("{query} raised an error: \n {exception}".format(
                query=query, exception=exception))
            raise

In [136]:
user = neo_user
password = neo_pass
uri = neo_uri
app = App(uri, user, password)

In [6]:

a = rumble_return("""let $a := for $matches in json-doc("matches_id.json")[]
for $match in $matches[]
for $doc in json-doc("lineups_cleaned/" || string($match) || ".json")
for $team in $doc[]
for $player in $team.lineup[]
return { "name" : $player.player_name, "player_id":$player.player_id, "jersey_number":$player.jersey_number, "country":$player.country.name}

for $player in $a
group by $player_name := $player.name, $player_id:= $player.player_id, $jersey_number := $player.jersey_number, $country := $player.country
return { "name" : $player_name, "player_id":$player_id, "jersey_number":$jersey_number, "country":$country}""")



In [103]:
def jsontoCypher(x):
    
    if type(x) is str:
        
        return '"'+x+'"'
    
    elif type(x) is dict:
        ret = []
        for key in x:
            ret.append(key+":"+jsontoCypher(x[key]))
        b = ", ".join(ret)
        return "{ " + b +" }"
     
    elif type(x) is list:
        vect = []
        for y in x:
            vect.append(jsontoCypher(y))
        return "["+", ".join(vect)+"]"
    elif type(x) is int or bool:
        
        return str(x)
    

In [119]:
js = json.loads(a)


for i in range(len(js)):
    x = js[i]
    args = ""
    args = "{"+ ", ".join([key+':"'+str(x[key]).replace('"',"'")+'"' for key in x])+"}"
    query = 'CREATE (x:PLAYER:'+x["country"].upper().replace(" ", "_").replace(",", "")+' '+jsontoCypher(x)+' ) RETURN x'
    app.create_player(query)

{name:"Michael Krmencik", player_id:"16037", jersey_number:"11", country:"Czech Republic"}
{ name:"Michael Krmencik", player_id:16037, jersey_number:11, country:"Czech Republic" }
CREATE (x:PLAYER:CZECH_REPUBLIC { name:"Michael Krmencik", player_id:16037, jersey_number:11, country:"Czech Republic" } ) RETURN x
Created player: {'': 'Michael Krmencik'}
{name:"Silvan Widmer", player_id:"7796", jersey_number:"3", country:"Switzerland"}
{ name:"Silvan Widmer", player_id:7796, jersey_number:3, country:"Switzerland" }
CREATE (x:PLAYER:SWITZERLAND { name:"Silvan Widmer", player_id:7796, jersey_number:3, country:"Switzerland" } ) RETURN x
Created player: {'': 'Silvan Widmer'}
{name:"lukasz Fabianski", player_id:"3262", jersey_number:"22", country:"Poland"}
{ name:"lukasz Fabianski", player_id:3262, jersey_number:22, country:"Poland" }
CREATE (x:PLAYER:POLAND { name:"lukasz Fabianski", player_id:3262, jersey_number:22, country:"Poland" } ) RETURN x
Created player: {'': 'lukasz Fabianski'}
{name:

Created player: {'': 'Stefan Ilsanker'}
{name:"Hakan calhanoglu", player_id:"7039", jersey_number:"10", country:"Turkey"}
{ name:"Hakan calhanoglu", player_id:7039, jersey_number:10, country:"Turkey" }
CREATE (x:PLAYER:TURKEY { name:"Hakan calhanoglu", player_id:7039, jersey_number:10, country:"Turkey" } ) RETURN x
Created player: {'': 'Hakan calhanoglu'}
{name:"Admir Mehmedi", player_id:"8219", jersey_number:"18", country:"Switzerland"}
{ name:"Admir Mehmedi", player_id:8219, jersey_number:18, country:"Switzerland" }
CREATE (x:PLAYER:SWITZERLAND { name:"Admir Mehmedi", player_id:8219, jersey_number:18, country:"Switzerland" } ) RETURN x
Created player: {'': 'Admir Mehmedi'}
{name:"Jan Bednarek", player_id:"4626", jersey_number:"5", country:"Poland"}
{ name:"Jan Bednarek", player_id:4626, jersey_number:5, country:"Poland" }
CREATE (x:PLAYER:POLAND { name:"Jan Bednarek", player_id:4626, jersey_number:5, country:"Poland" } ) RETURN x
Created player: {'': 'Jan Bednarek'}
{name:"Karol Line

Created player: {'': 'Thomas Mueller'}
{name:"Karim Onisiwo", player_id:"11278", jersey_number:"20", country:"Austria"}
{ name:"Karim Onisiwo", player_id:11278, jersey_number:20, country:"Austria" }
CREATE (x:PLAYER:AUSTRIA { name:"Karim Onisiwo", player_id:11278, jersey_number:20, country:"Austria" } ) RETURN x
Created player: {'': 'Karim Onisiwo'}
{name:"Glen Kamara", player_id:"13215", jersey_number:"6", country:"Finland"}
{ name:"Glen Kamara", player_id:13215, jersey_number:6, country:"Finland" }
CREATE (x:PLAYER:FINLAND { name:"Glen Kamara", player_id:13215, jersey_number:6, country:"Finland" } ) RETURN x
Created player: {'': 'Glen Kamara'}
{name:"Nathan Patterson", player_id:"37338", jersey_number:"22", country:"Scotland"}
{ name:"Nathan Patterson", player_id:37338, jersey_number:22, country:"Scotland" }
CREATE (x:PLAYER:SCOTLAND { name:"Nathan Patterson", player_id:37338, jersey_number:22, country:"Scotland" } ) RETURN x
Created player: {'': 'Nathan Patterson'}
{name:"Sergio Bus

Created player: {'': 'David Zima'}
{name:"Marek Rodak", player_id:"4152", jersey_number:"23", country:"Slovakia"}
{ name:"Marek Rodak", player_id:4152, jersey_number:23, country:"Slovakia" }
CREATE (x:PLAYER:SLOVAKIA { name:"Marek Rodak", player_id:4152, jersey_number:23, country:"Slovakia" } ) RETURN x
Created player: {'': 'Marek Rodak'}
{name:"Teemu Pukki", player_id:"9476", jersey_number:"10", country:"Finland"}
{ name:"Teemu Pukki", player_id:9476, jersey_number:10, country:"Finland" }
CREATE (x:PLAYER:FINLAND { name:"Teemu Pukki", player_id:9476, jersey_number:10, country:"Finland" } ) RETURN x
Created player: {'': 'Teemu Pukki'}
{name:"Joe Allen", player_id:"10931", jersey_number:"7", country:"Wales"}
{ name:"Joe Allen", player_id:10931, jersey_number:7, country:"Wales" }
CREATE (x:PLAYER:WALES { name:"Joe Allen", player_id:10931, jersey_number:7, country:"Wales" } ) RETURN x
Created player: {'': 'Joe Allen'}
{name:"Stanislav Lobotka", player_id:"6798", jersey_number:"22", countr

Created player: {'': 'Jason Denayer'}
{name:"Ilkay Guendogan", player_id:"10287", jersey_number:"21", country:"Germany"}
{ name:"Ilkay Guendogan", player_id:10287, jersey_number:21, country:"Germany" }
CREATE (x:PLAYER:GERMANY { name:"Ilkay Guendogan", player_id:10287, jersey_number:21, country:"Germany" } ) RETURN x
Created player: {'': 'Ilkay Guendogan'}
{name:"Kevin Volland", player_id:"8215", jersey_number:"9", country:"Germany"}
{ name:"Kevin Volland", player_id:8215, jersey_number:9, country:"Germany" }
CREATE (x:PLAYER:GERMANY { name:"Kevin Volland", player_id:8215, jersey_number:9, country:"Germany" } ) RETURN x
Created player: {'': 'Kevin Volland'}
{name:"Stefan Posch", player_id:"11821", jersey_number:"5", country:"Austria"}
{ name:"Stefan Posch", player_id:11821, jersey_number:5, country:"Austria" }
CREATE (x:PLAYER:AUSTRIA { name:"Stefan Posch", player_id:11821, jersey_number:5, country:"Austria" } ) RETURN x
Created player: {'': 'Stefan Posch'}
{name:"Robert Skov", player_

...

Created player: {'': 'Milan Badelj'}
{name:"Marcus Thuram", player_id:"2972", jersey_number:"26", country:"France"}
{ name:"Marcus Thuram", player_id:2972, jersey_number:26, country:"France" }
CREATE (x:PLAYER:FRANCE { name:"Marcus Thuram", player_id:2972, jersey_number:26, country:"France" } ) RETURN x
Created player: {'': 'Marcus Thuram'}
{name:"Stole Dimitrievski", player_id:"21397", jersey_number:"1", country:"Macedonia, Republic of"}
{ name:"Stole Dimitrievski", player_id:21397, jersey_number:1, country:"Macedonia, Republic of" }
CREATE (x:PLAYER:MACEDONIA_REPUBLIC_OF { name:"Stole Dimitrievski", player_id:21397, jersey_number:1, country:"Macedonia, Republic of" } ) RETURN x
Created player: {'': 'Stole Dimitrievski'}
{name:"Mathias Jensen", player_id:"18722", jersey_number:"24", country:"Denmark"}
{ name:"Mathias Jensen", player_id:18722, jersey_number:24, country:"Denmark" }
CREATE (x:PLAYER:DENMARK { name:"Mathias Jensen", player_id:18722, jersey_number:24, country:"Denmark" } )

Created player: {'': 'Fabian Lukas Schar'}
{name:"Jan Vertonghen", player_id:"3077", jersey_number:"5", country:"Belgium"}
{ name:"Jan Vertonghen", player_id:3077, jersey_number:5, country:"Belgium" }
CREATE (x:PLAYER:BELGIUM { name:"Jan Vertonghen", player_id:3077, jersey_number:5, country:"Belgium" } ) RETURN x
Created player: {'': 'Jan Vertonghen'}
{name:"Artem Besedin", player_id:"18881", jersey_number:"19", country:"Ukraine"}
{ name:"Artem Besedin", player_id:18881, jersey_number:19, country:"Ukraine" }
CREATE (x:PLAYER:UKRAINE { name:"Artem Besedin", player_id:18881, jersey_number:19, country:"Ukraine" } ) RETURN x
Created player: {'': 'Artem Besedin'}
{name:"Jakub Brabec", player_id:"27622", jersey_number:"4", country:"Czech Republic"}
{ name:"Jakub Brabec", player_id:27622, jersey_number:4, country:"Czech Republic" }
CREATE (x:PLAYER:CZECH_REPUBLIC { name:"Jakub Brabec", player_id:27622, jersey_number:4, country:"Czech Republic" } ) RETURN x
Created player: {'': 'Jakub Brabec'}

Created player: {'': 'Okay Yokuslu'}
{name:"Kieran Tierney", player_id:"10540", jersey_number:"6", country:"Scotland"}
{ name:"Kieran Tierney", player_id:10540, jersey_number:6, country:"Scotland" }
CREATE (x:PLAYER:SCOTLAND { name:"Kieran Tierney", player_id:10540, jersey_number:6, country:"Scotland" } ) RETURN x
Created player: {'': 'Kieran Tierney'}
{name:"Ricardo Ivan Rodriguez Araya", player_id:"5544", jersey_number:"13", country:"Switzerland"}
{ name:"Ricardo Ivan Rodriguez Araya", player_id:5544, jersey_number:13, country:"Switzerland" }
CREATE (x:PLAYER:SWITZERLAND { name:"Ricardo Ivan Rodriguez Araya", player_id:5544, jersey_number:13, country:"Switzerland" } ) RETURN x
Created player: {'': 'Ricardo Ivan Rodriguez Araya'}
{name:"Robin Quaison", player_id:"8552", jersey_number:"22", country:"Sweden"}
{ name:"Robin Quaison", player_id:8552, jersey_number:22, country:"Sweden" }
CREATE (x:PLAYER:SWEDEN { name:"Robin Quaison", player_id:8552, jersey_number:22, country:"Sweden" } ) 

Created player: {'': 'Jan Gregus'}
{name:"Domagoj Vida", player_id:"5468", jersey_number:"21", country:"Croatia"}
{ name:"Domagoj Vida", player_id:5468, jersey_number:21, country:"Croatia" }
CREATE (x:PLAYER:CROATIA { name:"Domagoj Vida", player_id:5468, jersey_number:21, country:"Croatia" } ) RETURN x
Created player: {'': 'Domagoj Vida'}
{name:"Aleksey Miranchuk", player_id:"6299", jersey_number:"15", country:"Russia"}
{ name:"Aleksey Miranchuk", player_id:6299, jersey_number:15, country:"Russia" }
CREATE (x:PLAYER:RUSSIA { name:"Aleksey Miranchuk", player_id:6299, jersey_number:15, country:"Russia" } ) RETURN x
Created player: {'': 'Aleksey Miranchuk'}
{name:"Joao Felix Sequeira", player_id:"12041", jersey_number:"23", country:"Portugal"}
{ name:"Joao Felix Sequeira", player_id:12041, jersey_number:23, country:"Portugal" }
CREATE (x:PLAYER:PORTUGAL { name:"Joao Felix Sequeira", player_id:12041, jersey_number:23, country:"Portugal" } ) RETURN x
Created player: {'': 'Joao Felix Sequeir

Created player: {'': 'James Forrest'}
{name:"Davy Klaassen", player_id:"4318", jersey_number:"14", country:"Netherlands"}
{ name:"Davy Klaassen", player_id:4318, jersey_number:14, country:"Netherlands" }
CREATE (x:PLAYER:NETHERLANDS { name:"Davy Klaassen", player_id:4318, jersey_number:14, country:"Netherlands" } ) RETURN x
Created player: {'': 'Davy Klaassen'}
{name:"Ben White", player_id:"22809", jersey_number:"22", country:"England"}
{ name:"Ben White", player_id:22809, jersey_number:22, country:"England" }
CREATE (x:PLAYER:ENGLAND { name:"Ben White", player_id:22809, jersey_number:22, country:"England" } ) RETURN x
Created player: {'': 'Ben White'}
{name:"Sam Johnstone", player_id:"9458", jersey_number:"23", country:"England"}
{ name:"Sam Johnstone", player_id:9458, jersey_number:23, country:"England" }
CREATE (x:PLAYER:ENGLAND { name:"Sam Johnstone", player_id:9458, jersey_number:23, country:"England" } ) RETURN x
Created player: {'': 'Sam Johnstone'}
{name:"Vladimir Darida", play

## Loading Matches

In [120]:
a = rumble_return("""
for $match in json-doc("matches.json")[]
return $match """)

In [143]:
js = json.loads(a)

for x in js:
    del x["competition"]
    del x["season"]
    del x["match_status"]
    del x["match_status_360"]
    del x["last_updated"]
    del x["last_updated_360"]
    del x["metadata"]
    x["teams"] = [x["home_team"]["home_team_name"], x["away_team"]["away_team_name"]]
    x["home_team_group"] = x["home_team"]["home_team_group"]
    x["away_team_group"] = x["away_team"]["away_team_group"]
    try:
        x["home_team_manager"] = x["home_team"]["managers"][0]["name"]
    except: 
        pass
    try:
        x["away_team_manager"] = x["away_team"]["managers"][0]["name"]
    except:
        pass
    x["competition_stage"] = x["competition_stage"]["name"]
    x["stadium"] = x["stadium"]["name"]
    x["referee"] = x["referee"]["name"]
    x["away_team"] = x["away_team"]["away_team_name"]
    x["home_team"] = x["home_team"]["home_team_name"]
    x["score"] = str(x["home_score"])+"-"+str(x["away_score"])
    
    query = 'CREATE (x:GAME' + ' '+jsontoCypher(x).replace("None","null")+' ) RETURN x'
    app.create_player(query)
    

Created player: {'': None}
Created player: {'': None}
Created player: {'': None}
Created player: {'': None}
Created player: {'': None}
Created player: {'': None}
Created player: {'': None}
Created player: {'': None}
Created player: {'': None}
Created player: {'': None}
Created player: {'': None}
Created player: {'': None}
Created player: {'': None}
Created player: {'': None}
Created player: {'': None}
Created player: {'': None}
Created player: {'': None}
Created player: {'': None}
Created player: {'': None}
Created player: {'': None}
Created player: {'': None}
Created player: {'': None}
Created player: {'': None}
Created player: {'': None}
Created player: {'': None}
Created player: {'': None}
Created player: {'': None}
Created player: {'': None}
Created player: {'': None}
Created player: {'': None}
Created player: {'': None}
Created player: {'': None}
Created player: {'': None}
Created player: {'': None}
Created player: {'': None}
Created player: {'': None}
Created player: {'': None}
C

## Create player---HAS_PLAYED-->game relationships

In [138]:

a = rumble_return("""
for $matches in json-doc("matches_id.json")[]
for $match in $matches[]
for $doc in json-doc("lineups_cleaned/" || string($match) || ".json")
for $team in $doc[]
for $player in $team.lineup[]
where size($player.positions) ge 1
return {"from": $player.player_name, "to": $match, "properties": $player}""")


In [139]:
js = json.loads(a)

for i in range(len(js)):
    y = js[i]
    x = y["properties"]
    
    del x["player_id"]
    del x["player_name"]
    del x["player_nickname"]
    del x["jersey_number"]
    del x["country"]
    
    if x["positions"][0]["start_reason"] == "Starting XI":
        x["starting"] = True
    else:
        x["starting"] = False
    
    p =[]
    for el in x["positions"]:
        if el["position"] is not None:
            p.append(el["position"])
    
    x["positions"] = p
    
    c =[]
    for el in x["cards"]:
        if el["card_type"] is not None:
            c.append(el["card_type"])
    
    x["cards"] = c
        
    args = ""
    args = "{"+ ", ".join([key+':"'+str(x[key]).replace('"',"'")+'"' for key in x])+"}"
    query = \
           'MATCH (a), (b) \
            WHERE a.name = "' + y["from"] + '" AND b.match_id = '+str(y["to"])+ \
            ' CREATE (a)-[r: HAS_PLAYED '+jsontoCypher(x).replace("None", "null")+']->(b) \
            RETURN r'
    app.create_relationship(query)

2022-04-03 15:32:21.850 ERROR   neo4j: Failed to write data to connection IPv4Address(('162.55.245.102', 7687)) (IPv4Address(('162.55.245.102', 7687)))


## Create Passages

In [57]:
def get_rumble_query(game, team):
    team = '"'+team+'"'
    return """
let $beg := for $doc in json-doc("events_cleaned/"""+str(game)+""".json")
    for $x in $doc[]
    count $c
    where $x."type".name eq "Pass" and $x.possession_team.name eq """ +team +""" and $x.team.name eq """ +team +"""
    where $x.pass.recipient.name
    return { "from" : $x.player.name, "to":$x.pass.recipient.name, "properties":$x, "possession":$x.possession
    }


    let $grouped := for $e in $beg
    group by $p:=$e.possession
    order by $p
    return {"possession":$p, "values":$e}

let $int:=for $g in $grouped
for $el in $g.values[]
order by $el.properties.period, $el.properties.timestamp
count $c
return {"el": $el, "count":$c}

let $scale := for $el in $int
group by $p := $el.el.possession
return {"possession":$p, "min":min($el.count), "max":max($el.count)+1-min($el.count)}

for $el in $int
for $p in $scale
where $p.possession eq $el.el.possession
return { "from" : $el.el.from, "to":$el.el.to,  "possession":$p.possession, "properties":$el.el.properties,
        "order":$el.count +1 - $p.min, "possession_length" : $p.max
    }"""

In [71]:
games = json.loads(rumble_return("""for $matches in json-doc("matches_id.json")[] for $match in $matches[] return $match"""))


for game in games:
    teams = json.loads(rumble_return("""for $match in json-doc("matches.json")[] 
    where $match.match_id eq """+str(game)+"""
    return ($match.home_team.home_team_name, $match.away_team.away_team_name)"""))
    
    print("Processing game "+ str(games.index(game)+1)+" / "+str(len(games)))
    
    for team in teams:
        
        passes = json.loads(rumble_return(get_rumble_query(game, team)))
        print("Processing "+team+" ("+str(len(passes))+" passages)")
        for passage in passes:
            x = passage["properties"]
            x["order"] = passage["order"]
            x["possession_length"] = passage["possession_length"]
            x["match_id"] = game 
            del x["id"]
            del x["type"]
            del x["possession_team"]
            del x["related_events"]
            del x["player"]
            try:
                x["team"] = x["team"]["name"]
            except:
                pass
            try:
                x["play_pattern"] = x["play_pattern"]["name"]
            except:
                pass
            try:
                x["position"] = x["position"]["name"]
            except:
                pass
            try:
                x["length"] = x["pass"]["length"]
            except:
                pass
            try:
                x["angle"] = x["pass"]["angle"]
            except:
                pass
            try:
                x["height"] = x["pass"]["height"]["name"]
            except:
                pass
            try:
                x["end_location"] = x["pass"]["end_location"]
            except:
                pass
            try:
                x["type"] = x["pass"]["type"]["name"]
            except:
                pass
            try:
                x["body_part"] = x["pass"]["body_part"]["name"]
            except:
                pass
            
            del x["pass"]
            
            
            
            query = \
           'MATCH (a), (b) \
            WHERE a.name = "' + passage["from"] + '" AND b.name = "'+passage["to"]+ \
            '" CREATE (a)-[r: PASS '+jsontoCypher(x)+']->(b) \
            RETURN type(r)'
            
            app.create_relationship(query)
        
    
    

Processing game 1 / 51
Processing Finland (364 passages)
Processing Russia (548 passages)
Processing game 2 / 51
Processing Switzerland (457 passages)
Processing Turkey (442 passages)
Processing game 3 / 51
Processing Belgium (416 passages)
Processing Italy (515 passages)
Processing game 4 / 51
Processing England (716 passages)
Processing Denmark (487 passages)
Processing game 5 / 51
Processing Italy (842 passages)
Processing England (394 passages)
Processing game 6 / 51
Processing England (413 passages)
Processing Germany (519 passages)
Processing game 7 / 51
Processing Sweden (626 passages)
Processing Ukraine (748 passages)
Processing game 8 / 51
Processing Croatia (415 passages)
Processing Spain (853 passages)
Processing game 9 / 51
Processing Belgium (411 passages)
Processing Portugal (577 passages)
Processing game 10 / 51
Processing Italy (661 passages)
Processing Austria (567 passages)
Processing game 11 / 51
Processing Germany (682 passages)
Processing Hungary (188 passages)
Pro

In [56]:
%%rumble
let $beg := for $doc in json-doc("events_cleaned/3788753.json")
    for $x in $doc[]
    count $c
    where $x."type".name eq "Pass" and $x.possession_team.name eq "Finland" and $x.team.name eq "Finland"
    where $x.pass.recipient.name
    return { "from" : $x.player.name, "to":$x.pass.recipient.name, "properties":$x, "possession":$x.possession
    }


    let $grouped := for $e in $beg
    group by $p:=$e.possession
    order by $p
    return {"possession":$p, "values":$e}

let $int:=for $g in $grouped
for $el in $g.values[]
order by $el.properties.period, $el.properties.timestamp
count $c
return {"el": $el, "count":$c}

let $scale := for $el in $int
group by $p := $el.el.possession
return {"possession":$p, "min":min($el.count), "max":max($el.count)+1-min($el.count)}

for $el in $int
for $p in $scale
where $p.possession eq $el.el.possession
return { "from" : $el.el.from, "to":$el.el.to,  "possession":$p.possession, "properties":$el.el.properties,
        "order":$el.count +1 - $p.min, "possession_length" : $p.max
    }

Took: 2.6685140132904053 s
{"from": "Paulus Arajuuri", "to": "Jukka Raitala", "possession": 3, "properties": {"id": "0457a0fa-f3f2-423f-a5ab-54d663fabb42", "index": 23, "period": 1, "timestamp": "00:00:14.780", "minute": 0, "second": 14, "type": {"id": 30, "name": "Pass"}, "possession": 3, "possession_team": {"id": 1835, "name": "Finland"}, "play_pattern": {"id": 1, "name": "Regular Play"}, "team": {"id": 1835, "name": "Finland"}, "player": {"id": 24455, "name": "Paulus Arajuuri"}, "position": {"id": 4, "name": "Center Back"}, "location": [4.1, 59.4], "duration": 0.848868, "related_events": ["3fa0546b-2424-4c0b-ba71-a14522dec67e"], "pass": {"recipient": {"id": 15416, "name": "Jukka Raitala"}, "length": 7.2249565, "angle": 1.6539376, "height": {"id": 1, "name": "Ground Pass"}, "end_location": [3.5, 66.6], "type": {"id": 66, "name": "Recovery"}, "body_part": {"id": 40, "name": "Right Foot"}}}, "order": 1, "possession_length": 2}

{"from": "Jukka Raitala", "to": "Rasmus Schueller", "posse

## Ending Events

In [105]:
query = """CREATE (X:SPECIAL:END {info:"dummy node to represent end of an action", type:"END"})"""



app.create_player(query)


In [106]:
def get_rumble_END(game):
    return """for $doc in json-doc("events_cleaned/"""+str(game)+""".json")
    for $x in $doc[]
    where $x.team.name eq $x.possession_team.name and $x."type".name = ("Dispossessed", "Shot", "Foul Won", "Foul Committed", "Miscontrol")
    return {"from": $x.player.name, "to": "END", "properties": $x, "label": $x."type".name}"""

In [107]:
games = json.loads(rumble_return("""for $matches in json-doc("matches_id.json")[] for $match in $matches[] return $match"""))

for game in games:
    
        print("Processing game "+ str(games.index(game)+1)+" / "+str(len(games)))
        events = json.loads(rumble_return(get_rumble_END(game)))
        for event in events:
            x = event["properties"]
            if "id" in x:
                del x["id"]
            if "related_events" in x:
                del x["related_events"]
            
            
            
            if "possession_team" in x:
                x["possession_team"] = x["possession_team"]["name"]
            if "play_pattern" in x:
                x["play_pattern"] = x["play_pattern"]["name"]
            del x["team"]
            del x["player"]
            del x["position"]
            
            if x["type"]["name"] == "Shot":
                y = x["shot"]
                if "statsbomb_xg" in y:
                    del y["statsbomb_xg"]
                if "key_pass_id" in y: 
                    del y["key_pass_id"]
                if "freeze_frame" in y:
                    del y["freeze_frame"]
                if "end_location" in y:
                    x["shot_end_location"] = y["end_location"]
                if "type" in y:
                    x["shot_type"] = y["type"]["name"]
                if "body_part" in y:
                    x["body_part"] = y["body_part"]["name"]
                if "technique" in y:
                    x["technique"] = y["technique"]["name"]
                
                del x["shot"]
                x["outcome"] = y["outcome"]["name"]
                
            if x["type"]["name"] == "Foul Committed":
                if "foul_committed" in x:
                    if "card" in x["foul_committed"]:
                        x["card"] = x["foul_committed"]["card"]["name"]
                    
                    del x["foul_committed"]
        
            if "foul_won" in x:
                del x["foul_won"]
                
            if "miscontrol" in x:
                del x["miscontrol"]
            x["type"] = x["type"]["name"]
            x["match_id"] = game
            
            query = \
               'MATCH (a), (b:END) \
                WHERE a.name = "' + event["from"] + '" AND b.type = "'+str(event["to"])+ \
                '" CREATE (a)-[r:'+event["label"].replace(" ", "_").upper()+" "+jsontoCypher(x).replace("None", "null")+']->(b) \
                RETURN type(r)'
            #print(query)
            
            app.create_relationship(query)
        
        
            

Processing game 1 / 51
Processing game 2 / 51
Processing game 3 / 51
Processing game 4 / 51
Processing game 5 / 51
Processing game 6 / 51
Processing game 7 / 51
Processing game 8 / 51
Processing game 9 / 51
Processing game 10 / 51
Processing game 11 / 51
Processing game 12 / 51
Processing game 13 / 51
Processing game 14 / 51
Processing game 15 / 51
Processing game 16 / 51
Processing game 17 / 51
Processing game 18 / 51
Processing game 19 / 51
Processing game 20 / 51
Processing game 21 / 51
Processing game 22 / 51
Processing game 23 / 51
Processing game 24 / 51
Processing game 25 / 51
Processing game 26 / 51
Processing game 27 / 51
Processing game 28 / 51
Processing game 29 / 51
Processing game 30 / 51
Processing game 31 / 51
Processing game 32 / 51
Processing game 33 / 51
Processing game 34 / 51
Processing game 35 / 51
Processing game 36 / 51
Processing game 37 / 51
Processing game 38 / 51
Processing game 39 / 51
Processing game 40 / 51
Processing game 41 / 51
Processing game 42 / 51
P

## Starting Events

In [117]:
def get_rumble_query_START(game):
    return """for $doc in json-doc("events_cleaned/"""+str(game)+""".json")
    for $x in $doc[]
    where $x.team.name eq $x.possession_team.name and $x."type".name = ("Ball Recovery")
    return {"to": $x.player.name, "from": "START", "properties": $x, "label": $x."type".name}"""

In [118]:
#query = """CREATE (X:SPECIAL:START {info:"dummy node to represent the beginning of an action", type:"START"})"""

#app.create_player(query)


In [122]:
games = json.loads(rumble_return("""for $matches in json-doc("matches_id.json")[] for $match in $matches[] return $match"""))
for game in games:
    
        print("Processing game "+ str(games.index(game)+1)+" / "+str(len(games)))
        events = json.loads(rumble_return(get_rumble_query_START(game)))
        for event in events:
            
            x = event["properties"]
            if "id" in x:
                del x["id"]
            if "related_events" in x:
                del x["related_events"]

            x["match_id"] = game
            x["type"] = x["type"]["name"]
            x["possession_team"] = x["possession_team"]["name"]
            if "play_pattern" in x:
                x["play_pattern"] = x["play_pattern"]["name"]
            del x["team"]
            
            if "player" in x:
                del x["player"]
            if "position" in x:
                del x["position"]
            if "ball_recovery" in x:
                del x["ball_recovery"]
            
            
            query = \
               'MATCH (a:START), (b) \
                WHERE a.type = "' + event["from"] + '" AND b.name = "'+str(event["to"])+ \
                '" CREATE (a)-[r: '+event["label"].replace(" ", "_").replace("-","_").upper()+" "+jsontoCypher(x).replace("None", "null")+']->(b) \
                RETURN type(r)'
            
            
            app.create_relationship(query)
       
        
        

Processing game 1 / 51
Processing game 2 / 51
Processing game 3 / 51
Processing game 4 / 51
Processing game 5 / 51
Processing game 6 / 51
Processing game 7 / 51
Processing game 8 / 51
Processing game 9 / 51
Processing game 10 / 51
Processing game 11 / 51
Processing game 12 / 51
Processing game 13 / 51
Processing game 14 / 51
Processing game 15 / 51
Processing game 16 / 51
Processing game 17 / 51
Processing game 18 / 51
Processing game 19 / 51
Processing game 20 / 51
Processing game 21 / 51
Processing game 22 / 51
Processing game 23 / 51
Processing game 24 / 51
Processing game 25 / 51
Processing game 26 / 51
Processing game 27 / 51
Processing game 28 / 51
Processing game 29 / 51
Processing game 30 / 51
Processing game 31 / 51
Processing game 32 / 51
Processing game 33 / 51
Processing game 34 / 51
Processing game 35 / 51
Processing game 36 / 51
Processing game 37 / 51
Processing game 38 / 51
Processing game 39 / 51
Processing game 40 / 51
Processing game 41 / 51
Processing game 42 / 51
P

## Old

In [123]:
l = [
            {
                "n.home_team": "Finland",
                "n.away_team": "Russia",
                "n.match_id": 3788753
            },
            {
                "n.home_team": "Switzerland",
                "n.away_team": "Turkey",
                "n.match_id": 3788765
            },
            {
                "n.home_team": "Belgium",
                "n.away_team": "Italy",
                "n.match_id": 3795107
            },
            {
                "n.home_team": "England",
                "n.away_team": "Denmark",
                "n.match_id": 3795221
            },
            {
                "n.home_team": "Italy",
                "n.away_team": "England",
                "n.match_id": 3795506
            },
            {
                "n.home_team": "England",
                "n.away_team": "Germany",
                "n.match_id": 3794688
            },
            {
                "n.home_team": "Sweden",
                "n.away_team": "Ukraine",
                "n.match_id": 3794692
            },
            {
                "n.home_team": "Croatia",
                "n.away_team": "Spain",
                "n.match_id": 3794686
            },
            {
                "n.home_team": "Belgium",
                "n.away_team": "Portugal",
                "n.match_id": 3794687
            },
            {
                "n.home_team": "Italy",
                "n.away_team": "Austria",
                "n.match_id": 3794685
            },
            {
                "n.home_team": "Germany",
                "n.away_team": "Hungary",
                "n.match_id": 3788774
            },
            {
                "n.home_team": "Croatia",
                "n.away_team": "Scotland",
                "n.match_id": 3788771
            },
            {
                "n.home_team": "Czech Republic",
                "n.away_team": "England",
                "n.match_id": 3788772
            },
            {
                "n.home_team": "Finland",
                "n.away_team": "Belgium",
                "n.match_id": 3788768
            },
            {
                "n.home_team": "Ukraine",
                "n.away_team": "Austria",
                "n.match_id": 3788767
            },
            {
                "n.home_team": "Hungary",
                "n.away_team": "France",
                "n.match_id": 3788763
            },
            {
                "n.home_team": "England",
                "n.away_team": "Scotland",
                "n.match_id": 3788759
            },
            {
                "n.home_team": "Ukraine",
                "n.away_team": "North Macedonia",
                "n.match_id": 3788758
            },
            {
                "n.home_team": "Denmark",
                "n.away_team": "Belgium",
                "n.match_id": 3788757
            },
            {
                "n.home_team": "England",
                "n.away_team": "Croatia",
                "n.match_id": 3788745
            },
            {
                "n.home_team": "Netherlands",
                "n.away_team": "Ukraine",
                "n.match_id": 3788746
            },
            {
                "n.home_team": "France",
                "n.away_team": "Switzerland",
                "n.match_id": 3794691
            },
            {
                "n.home_team": "Netherlands",
                "n.away_team": "Czech Republic",
                "n.match_id": 3794690
            },
            {
                "n.home_team": "Wales",
                "n.away_team": "Denmark",
                "n.match_id": 3794689
            },
            {
                "n.home_team": "Russia",
                "n.away_team": "Denmark",
                "n.match_id": 3788769
            },
            {
                "n.home_team": "Sweden",
                "n.away_team": "Slovakia",
                "n.match_id": 3788761
            },
            {
                "n.home_team": "Portugal",
                "n.away_team": "Germany",
                "n.match_id": 3788764
            },
            {
                "n.home_team": "Italy",
                "n.away_team": "Wales",
                "n.match_id": 3788766
            },
            {
                "n.home_team": "Italy",
                "n.away_team": "Switzerland",
                "n.match_id": 3788754
            },
            {
                "n.home_team": "Turkey",
                "n.away_team": "Wales",
                "n.match_id": 3788755
            },
            {
                "n.home_team": "Portugal",
                "n.away_team": "France",
                "n.match_id": 3788773
            },
            {
                "n.home_team": "Spain",
                "n.away_team": "Poland",
                "n.match_id": 3788762
            },
            {
                "n.home_team": "Croatia",
                "n.away_team": "Czech Republic",
                "n.match_id": 3788760
            },
            {
                "n.home_team": "Austria",
                "n.away_team": "North Macedonia",
                "n.match_id": 3788747
            },
            {
                "n.home_team": "Turkey",
                "n.away_team": "Italy",
                "n.match_id": 3788741
            },
            {
                "n.home_team": "Italy",
                "n.away_team": "Spain",
                "n.match_id": 3795220
            },
            {
                "n.home_team": "North Macedonia",
                "n.away_team": "Netherlands",
                "n.match_id": 3788770
            },
            {
                "n.home_team": "Switzerland",
                "n.away_team": "Spain",
                "n.match_id": 3795108
            },
            {
                "n.home_team": "Ukraine",
                "n.away_team": "England",
                "n.match_id": 3795187
            },
            {
                "n.home_team": "Czech Republic",
                "n.away_team": "Denmark",
                "n.match_id": 3795109
            },
            {
                "n.home_team": "Netherlands",
                "n.away_team": "Austria",
                "n.match_id": 3788756
            },
            {
                "n.home_team": "Poland",
                "n.away_team": "Slovakia",
                "n.match_id": 3788749
            },
            {
                "n.home_team": "Spain",
                "n.away_team": "Sweden",
                "n.match_id": 3788750
            },
            {
                "n.home_team": "Scotland",
                "n.away_team": "Czech Republic",
                "n.match_id": 3788748
            },
            {
                "n.home_team": "France",
                "n.away_team": "Germany",
                "n.match_id": 3788751
            },
            {
                "n.home_team": "Hungary",
                "n.away_team": "Portugal",
                "n.match_id": 3788752
            },
            {
                "n.home_team": "Denmark",
                "n.away_team": "Finland",
                "n.match_id": 3788742
            },
            {
                "n.home_team": "Slovakia",
                "n.away_team": "Spain",
                "n.match_id": 3788775
            },
            {
                "n.home_team": "Sweden",
                "n.away_team": "Poland",
                "n.match_id": 3788776
            },
            {
                "n.home_team": "Belgium",
                "n.away_team": "Russia",
                "n.match_id": 3788743
            },
            {
                "n.home_team": "Wales",
                "n.away_team": "Switzerland",
                "n.match_id": 3788744
            }
        ]

In [131]:
teams = set()
for el in l:
    teams.add(el["n.home_team"])
    teams.add(el["n.away_team"])

teams

{'Austria',
 'Belgium',
 'Croatia',
 'Czech Republic',
 'Denmark',
 'England',
 'Finland',
 'France',
 'Germany',
 'Hungary',
 'Italy',
 'Netherlands',
 'North Macedonia',
 'Poland',
 'Portugal',
 'Russia',
 'Scotland',
 'Slovakia',
 'Spain',
 'Sweden',
 'Switzerland',
 'Turkey',
 'Ukraine',
 'Wales'}

In [4]:
%%rumble
distinct-values(for $doc in json-doc("3788757.json")
for $team in $doc[]
where $team.team_name eq "Denmark"
for $player in $team.lineup[]
for $pos in $player.positions[]
return $player.player_name)

Took: 3.397970199584961 s
"Pierre-Emile Hojbjerg"
"Kasper Schmeichel"
"Andreas Christensen"
"Martin Braithwaite Christensen"
"Jens Stryger Larsen"
"Thomas Delaney"
"Simon Thorup Kjaer"
"Yussuf Yurary Poulsen"
"Andreas Evald Cornelius"
"Daniel Wass"
"Christian Norgaard"
"Jannik Vestergaard"
"Joakim Maehle"
"Andreas Skov Olsen"
"Mathias Jensen"
"Mikkel Damsgaard"


In [12]:
%%rumble
let $players := for $doc in json-doc("3788757.json")
for $team in $doc[]
where $team.team_name eq "Belgium"
for $player in $team.lineup[]
for $pos in $player.positions[]
return {"name":$player.player_name,
        "jersey_number":$player.jersey_number,
        "cards":$player.cards
        , "pos" : $pos}

let $a:=[for $player in $players
return {"name":$player.name,
    "jersey_number":$player.jersey_number,
    "cards":$player.cards,
    "start" : $player.pos.from,
    "end" : $player.pos.to,
    "pos" : $player.pos}]

let $multiple:=for $p in $a[]
group by $pl:=$p.name
where count($p.name) gt 1
let $all := {"name" : $p.name,
        "jersey_number":$p.jersey_number,
        "cards":$p.cards,
        "start" : $p.start,
        "end": $p.end,
        "pos" : $p.pos}
for $x in $all
return {
    "name": $x.name[[1]],
    "jersey_number":$x.jersey_number[[1]],
    "cards":$x.cards[[1]],
    "start" : $x.start[[1]],
    "end" : $x.end[[count($p.name)]],
    "pos" : $x.pos[[1]]}

let $single := for $p in $a[]
group by $pl:=$p.name
where count($p.name) eq 1
return $p

let $starters := for $p in ($single, $multiple)
where $p.pos.start_reason eq "Starting XI"
return {"name" : $p.name,
        "jersey_number":$p.jersey_number,
        "cards":$p.cards,
        "start" : $p.start,
        "end": $p.end,
        "starting" : true}

let $notstarters :=for $p in ($single, $multiple)
where $p.pos.start_reason ne "Starting XI"
return {"name" : $p.name,
        "jersey_number":$p.jersey_number,
        "cards":$p.cards,
        "start" : $p.start,
        "end": $p.end,
        "starting" : false}

return ($notstarters, $starters)



let $players := for $doc in json-doc("3788757.json")
for $team in $doc[]
where $team.team_name eq "Belgium"
for $player in $team.lineup[]
for $pos in $player.positions[]
return {"name":$player.player_name,
        "jersey_number":$player.jersey_number,
        "cards":$player.cards
        , "pos" : $pos}

let $a:=[for $player in $players
return {"name":$player.name,
    "jersey_number":$player.jersey_number,
    "cards":$player.cards,
    "start" : $player.pos.from,
    "end" : $player.pos.to,
    "pos" : $player.pos}]

let $multiple:=for $p in $a[]
group by $pl:=$p.name
where count($p.name) gt 1
let $all := {"name" : $p.name,
        "jersey_number":$p.jersey_number,
        "cards":$p.cards,
        "start" : $p.start,
        "end": $p.end,
        "pos" : $p.pos}
for $x in $all
return {
    "name": $x.name[[1]],
    "jersey_number":$x.jersey_number[[1]],
    "cards":$x.cards[[1]],
    "start" : $x.start[[1]],
    "end" : $x.end[[count($p.name)]],
    "pos" : $x.pos[[1]]}

let $sin

In [8]:
%%rumble
let $players := for $doc in json-doc("3788757.json")
for $team in $doc[]
where $team.team_name eq "Belgium"
for $player in $team.lineup[]
for $pos in $player.positions[]
return {"name":$player.player_name,
        "jersey_number":$player.jersey_number,
        "cards":$player.cards, "pos" : $pos}

let $a:=[for $player in $players
return {"name":$player.name,
    "jersey_number":$player.jersey_number,
    "cards":$player.cards,
    "start" : $player.pos.from,
    "end" : $player.pos.to,
    "pos" : $player.pos}]

let $multiple:=for $p in $a[]
group by $pl:=$p.name
where count($p.name) gt 1
let $all := {"name" : $p.name,
        "jersey_number":$p.jersey_number,
        "cards":$p.cards,
        "start" : $p.start,
        "end": $p.end,
        "pos" : $p.pos}
for $x in $all
return {
    "name": $x.name[[1]],
    "jersey_number":$x.jersey_number[[1]],
    "cards":$x.cards[[1]],
    "start" : $x.start[[1]],
    "end" : $x.end[[count($p.name)]],
    "pos" : $x.pos[[1]]}

let $single := for $p in $a[]
group by $pl:=$p.name
where count($p.name) eq 1
return $p

let $starters := for $p in ($single, $multiple)
where $p.pos.start_reason eq "Starting XI"
return {"name" : $p.name,
        "jersey_number":$p.jersey_number,
        "cards":$p.cards,
        "start" : $p.start,
        "end": $p.end,
        "starting" : true}

let $notstarters :=for $p in ($single, $multiple)
where $p.pos.start_reason ne "Starting XI"
return {"name" : $p.name,
        "jersey_number":$p.jersey_number,
        "cards":$p.cards,
        "start" : $p.start,
        "end": $p.end,
        "starting" : false}

return ($notstarters, $starters)


Took: 2.173999547958374 s
{"name": "Eden Hazard", "jersey_number": 10, "cards": [], "start": "58:27", "end": null, "starting": false}

{"name": "Kevin De Bruyne", "jersey_number": 7, "cards": [], "start": "45:00", "end": null, "starting": false}

{"name": "Thomas Vermaelen", "jersey_number": 3, "cards": [], "start": "93:52", "end": null, "starting": false}

{"name": "Axel Witsel", "jersey_number": 6, "cards": [], "start": "58:29", "end": null, "starting": false}

{"name": "Thorgan Hazard", "jersey_number": 16, "cards": [{"time": "93:19", "card_type": "Yellow Card", "reason": "Foul Committed", "period": 2}], "start": "00:00", "end": "93:52", "starting": true}

{"name": "Jason Denayer", "jersey_number": 18, "cards": [], "start": "00:00", "end": null, "starting": true}

{"name": "Jan Vertonghen", "jersey_number": 5, "cards": [], "start": "00:00", "end": null, "starting": true}

{"name": "Thibaut Courtois", "jersey_number": 1, "cards": [], "start": "00:00", "end": null, "starting": true}



In [61]:
%%rumble
for $doc in json-doc("3788757.json")
for $team in $doc[]
where $team.team_name eq "Belgium"


Took: 2.0905919075012207 s
{"name": "Youri Tielemans", "pos": {"position_id": 11, "position": "Left Defensive Midfield", "from": "00:00", "to": "59:32", "from_period": 1, "to_period": 2, "start_reason": "Starting XI", "end_reason": "Tactical Shift"}}
{"name": "Youri Tielemans", "pos": {"position_id": 9, "position": "Right Defensive Midfield", "from": "59:32", "to": null, "from_period": 2, "to_period": null, "start_reason": "Tactical Shift", "end_reason": "Final Whistle"}}
{"name": "Jan Vertonghen", "pos": {"position_id": 5, "position": "Left Center Back", "from": "00:00", "to": null, "from_period": 1, "to_period": null, "start_reason": "Starting XI", "end_reason": "Final Whistle"}}
{"name": "Kevin De Bruyne", "pos": {"position_id": 18, "position": "Right Attacking Midfield", "from": "45:00", "to": null, "from_period": 2, "to_period": null, "start_reason": "Substitution - On (Tactical)", "end_reason": "Final Whistle"}}
{"name": "Thomas Meunier", "pos": {"position_id": 7, "position": "Ri

Took: 2.407909631729126 s


'There was an error on line 17 in file:/C:/User/franc/:\n\nreturn index-of($e, $x)\n                ^\n\nCode: [XPTY0004]\nMessage: Invalid argument for function index-of. js:object cannot be promoted to type js:atomic*.\nMetadata: file:/C:/User/franc/:LINE:17:COLUMN:16:\nThis code can also be looked up in the documentation and specifications for more information.\n'

In [8]:
%%rumble
let $beg := for $doc in json-doc("3788757_events.json")
for $x in $doc[]
count $c
where $x."type".name eq "Pass" and $x.possession_team.name eq "Belgium" and $x.team.name eq "Belgium"
where $x.pass.recipient.name
return { "from" : $x.player.name, "to":$x.pass.recipient.name, "possession" : $x.possession, "length": $x.pass.length, "angle": 
        $x.pass.angle, "height":$x.pass.height.name, "timestamp": $x.timestamp, "pattern" : $x.play_pattern.name, 
 "body_part":$x.pass.body_part.name, "period": $x.period, "location":$x.location
}


for $e in $beg
group by $p:=$e.possession
for $x in $e
order by $x.period, $x.timestamp
count $c
return { "from" : $x.from, "to":$x.to, "possession" : $x.possession, "length": $x.length, "angle": 
        $x.angle, "height":$x.height, "timestamp": $x.timestamp, "pattern" : $x.pattern, "order": $c, 
 "body_part":$x.body_part, "period":$x.period, "location":$x.location
}


Took: 2.9698662757873535 s
{"from": "Thorgan Hazard", "to": "Jan Vertonghen", "possession": 4, "length": 13.857128, "angle": 2.6036484, "height": "Low Pass", "timestamp": "00:00:39.705", "pattern": "From Throw In", "order": 1, "body_part": null, "period": 1, "location": [35.5, 0.1]}
{"from": "Jan Vertonghen", "to": "Yannick Ferreira Carrasco", "possession": 4, "length": 40.30943, "angle": 0.24558035, "height": "High Pass", "timestamp": "00:00:40.666", "pattern": "From Throw In", "order": 2, "body_part": "Left Foot", "period": 1, "location": [22.7, 7.4]}
{"from": "Thibaut Courtois", "to": "Jason Denayer", "possession": 5, "length": 7.3334846, "angle": -1.4751981, "height": "Ground Pass", "timestamp": "00:00:57.802", "pattern": "From Keeper", "order": 3, "body_part": "Keeper Arm", "period": 1, "location": [8.8, 45.3]}
{"from": "Jason Denayer", "to": "Jan Vertonghen", "possession": 5, "length": 26.271849, "angle": -1.456356, "height": "Ground Pass", "timestamp": "00:00:59.748", "pattern":

In [45]:
%%rumble
for $doc in json-doc("3788757_events.json")
for $x in $doc[]
where $x."type".name ne "Pass"
group by $t:= $x."type".name
order by $t
return {"type":$t, "count": count($x)}



Took: 2.325173854827881 s
{"type": "Ball Receipt*", "count": 1105}

{"type": "Ball Recovery", "count": 75}

{"type": "Block", "count": 28}

{"type": "Carry", "count": 1005}

{"type": "Clearance", "count": 30}

{"type": "Dispossessed", "count": 18}

{"type": "Dribble", "count": 37}

{"type": "Dribbled Past", "count": 25}

{"type": "Duel", "count": 62}

{"type": "Error", "count": 1}

{"type": "Foul Committed", "count": 34}

{"type": "Foul Won", "count": 32}

{"type": "Goal Keeper", "count": 34}

{"type": "Half End", "count": 4}

{"type": "Half Start", "count": 4}

{"type": "Injury Stoppage", "count": 3}

{"type": "Interception", "count": 36}

{"type": "Miscontrol", "count": 20}

{"type": "Pressure", "count": 258}

{"type": "Referee Ball-Drop", "count": 3}

{"type": "Shield", "count": 1}

{"type": "Shot", "count": 27}

{"type": "Starting XI", "count": 2}

{"type": "Substitution", "count": 9}

{"type": "Tactical Shift", "count": 4}



In [6]:
%%rumble
for $doc in json-doc("3788757_events.json")
for $x in $doc[]
where $x."type".name eq "Ball Receipt*"
return $x

Took: 2.687516212463379 s
"Warning! The output sequence contains 9223372036854775807 items but its materialization was capped at 1000 items. This value can be configured with the result-size parameter in the query string of the HTTP request."
{"id": "435afe1f-6f3b-46a7-98d6-720e0825319d", "index": 6, "period": 1, "timestamp": "00:00:02.592", "minute": 0, "second": 2, "type": {"id": 42, "name": "Ball Receipt*"}, "possession": 2, "possession_team": {"id": 776, "name": "Denmark"}, "play_pattern": {"id": 9, "name": "From Kick Off"}, "team": {"id": 776, "name": "Denmark"}, "player": {"id": 8815, "name": "Jannik Vestergaard"}, "position": {"id": 5, "name": "Left Center Back"}, "location": [40.7, 35], "related_events": ["c2676711-e363-48e9-bbd9-6572bb65438f"]}

{"id": "1de3cd93-7d20-4aef-a852-7bc9953f15cb", "index": 9, "period": 1, "timestamp": "00:00:08.612", "minute": 0, "second": 8, "type": {"id": 42, "name": "Ball Receipt*"}, "possession": 2, "possession_team": {"id": 776, "name": "Denmar

{"id": "dd79d91c-8caa-4fe4-b15e-7300a2f026a8", "index": 2926, "period": 2, "timestamp": "00:23:33.084", "minute": 68, "second": 33, "type": {"id": 42, "name": "Ball Receipt*"}, "possession": 141, "possession_team": {"id": 776, "name": "Denmark"}, "play_pattern": {"id": 1, "name": "Regular Play"}, "team": {"id": 776, "name": "Denmark"}, "player": {"id": 5527, "name": "Thomas Delaney"}, "position": {"id": 11, "name": "Left Defensive Midfield"}, "location": [71.7, 32.8], "related_events": ["aab8e851-8419-4963-86b2-fa088332b7fc"]}

{"id": "a4199f83-d2d7-4e69-8689-a780411316e2", "index": 2928, "period": 2, "timestamp": "00:23:34.339", "minute": 68, "second": 34, "type": {"id": 42, "name": "Ball Receipt*"}, "possession": 141, "possession_team": {"id": 776, "name": "Denmark"}, "play_pattern": {"id": 1, "name": "Regular Play"}, "team": {"id": 776, "name": "Denmark"}, "player": {"id": 24443, "name": "Mikkel Damsgaard"}, "position": {"id": 20, "name": "Left Attacking Midfield"}, "location": [88.

In [43]:
%%rumble
for $doc in json-doc("3788757_events.json")
for $i in (27)
for $x in $doc[]
where $x.possession eq $i and $x.team.id eq 782
order by $x.timestamp
return [$i, $x."type".name, $x.player.name, $x.pass.recipient.name, $x.play_pattern.name]

Took: 2.3332183361053467 s
[27, "Pass", "Leander Dendoncker", "Thomas Meunier", "Regular Play"]

[27, "Ball Receipt*", "Thomas Meunier", "Regular Play"]

[27, "Carry", "Thomas Meunier", "Regular Play"]

[27, "Pass", "Thomas Meunier", "Dries Mertens", "Regular Play"]

[27, "Ball Receipt*", "Dries Mertens", "Regular Play"]

[27, "Carry", "Dries Mertens", "Regular Play"]

[27, "Pass", "Dries Mertens", "Romelu Lukaku Menama", "Regular Play"]

[27, "Ball Receipt*", "Romelu Lukaku Menama", "Regular Play"]

[27, "Carry", "Romelu Lukaku Menama", "Regular Play"]

[27, "Miscontrol", "Romelu Lukaku Menama", "Regular Play"]



In [49]:
%%rumble
for $doc in json-doc("3788757_events.json")
for $x in $doc[]
where $x.team.name eq "Belgium" and $x."type".name eq "Shot" and $x.possession_team.name eq "Belgium"
return $x

Took: 2.2207982540130615 s
{"id": "ccc77c39-cb4b-47ca-8d6b-063bda42674d", "index": 99, "period": 1, "timestamp": "00:03:07.814", "minute": 3, "second": 7, "type": {"id": 16, "name": "Shot"}, "possession": 9, "possession_team": {"id": 782, "name": "Belgium"}, "play_pattern": {"id": 9, "name": "From Kick Off"}, "team": {"id": 782, "name": "Belgium"}, "player": {"id": 5630, "name": "Dries Mertens"}, "position": {"id": 18, "name": "Right Attacking Midfield"}, "location": [112.5, 33.6], "duration": 0.468611, "related_events": ["67659d24-c33e-488a-a069-e186d080b3dc"], "shot": {"one_on_one": true, "statsbomb_xg": 0.1098275, "end_location": [119.4, 37.6, 0.3], "key_pass_id": "b15dd971-cb17-4f5b-8cd7-6dfa0ce4854e", "type": {"id": 87, "name": "Open Play"}, "body_part": {"id": 37, "name": "Head"}, "technique": {"id": 93, "name": "Normal"}, "outcome": {"id": 100, "name": "Saved"}, "freeze_frame": [{"location": [99.5, 51.6], "player": {"id": 3176, "name": "Thomas Meunier"}, "position": {"id": 7, "n

In [117]:
%%rumble
for $doc in json-doc("3788757_events.json")
for $x in $doc[]
where $x.team.name eq $x.possession_team.name and $x."type".name = ("Dispossessed", "Shot", "Foul Won", "Foul Committed", "Miscontrol")
return {"from": $x.player.name, "to": "END", "properties": $x, "label":$x."type".name}

Took: 2.3409855365753174 s
{"from": "Yannick Ferreira Carrasco", "to": "END", "properties": {"id": "c62d9b61-f604-49c3-a137-d59b5b7581cf", "index": 43, "period": 1, "timestamp": "00:01:08.817", "minute": 1, "second": 8, "type": {"id": 3, "name": "Dispossessed"}, "possession": 5, "possession_team": {"id": 782, "name": "Belgium"}, "play_pattern": {"id": 8, "name": "From Keeper"}, "team": {"id": 782, "name": "Belgium"}, "player": {"id": 5633, "name": "Yannick Ferreira Carrasco"}, "position": {"id": 20, "name": "Left Attacking Midfield"}, "location": [69.6, 11.1], "duration": 0, "under_pressure": true, "related_events": ["5577289d-d6b2-4263-9c75-31563352d162"]}, "label": "Dispossessed"}

{"from": "Yussuf Yurary Poulsen", "to": "END", "properties": {"id": "2fba73ef-d0bd-49fb-be0f-da32dc92f5d3", "index": 72, "period": 1, "timestamp": "00:01:39.893", "minute": 1, "second": 39, "type": {"id": 16, "name": "Shot"}, "possession": 8, "possession_team": {"id": 776, "name": "Denmark"}, "play_pattern

In [124]:
%%rumble
for $doc in json-doc("3788757_events.json")
    for $x in $doc[]
    where $x.team.name eq $x.possession_team.name and $x."type".name = ("Dispossessed", "Shot", "Foul Won", "Foul Committed", "Miscontrol")
    return {"from": $x.player.name, "to": "END", "properties": $x, "label": $x."type".name}

Took: 2.212371349334717 s
{"from": "Yannick Ferreira Carrasco", "to": "END", "properties": {"id": "c62d9b61-f604-49c3-a137-d59b5b7581cf", "index": 43, "period": 1, "timestamp": "00:01:08.817", "minute": 1, "second": 8, "type": {"id": 3, "name": "Dispossessed"}, "possession": 5, "possession_team": {"id": 782, "name": "Belgium"}, "play_pattern": {"id": 8, "name": "From Keeper"}, "team": {"id": 782, "name": "Belgium"}, "player": {"id": 5633, "name": "Yannick Ferreira Carrasco"}, "position": {"id": 20, "name": "Left Attacking Midfield"}, "location": [69.6, 11.1], "duration": 0, "under_pressure": true, "related_events": ["5577289d-d6b2-4263-9c75-31563352d162"]}, "label": "Dispossessed"}

{"from": "Yussuf Yurary Poulsen", "to": "END", "properties": {"id": "2fba73ef-d0bd-49fb-be0f-da32dc92f5d3", "index": 72, "period": 1, "timestamp": "00:01:39.893", "minute": 1, "second": 39, "type": {"id": 16, "name": "Shot"}, "possession": 8, "possession_team": {"id": 776, "name": "Denmark"}, "play_pattern"